In [21]:
#Importar librerias
import pandas as pd

In [22]:
# Leer dataset original
data = pd.read_csv("casos_covid19_colombia.csv")[[
    "ID de caso",
    "Nombre departamento",
    "Nombre municipio",
    "Edad",
    "Unidad de medida de edad",
    "Sexo",
    "Tipo de contagio",
    "Estado",
    "Fecha de inicio de síntomas"]]

data.shape

C:\Users\crist\AppData\Local\Temp\ipykernel_3848\226723303.py:2: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("casos_covid19_colombia.csv")[[


(6390971, 9)

# Preparación de los datos

In [23]:
# Selección de datos relevantes
clean_data = data[[
    "ID de caso",
    "Nombre departamento",
    "Nombre municipio",
    "Edad",
    "Unidad de medida de edad",
    "Sexo",
    "Tipo de contagio",
    "Estado",
    "Fecha de inicio de síntomas"]]

clean_data.shape

(6390971, 9)

In [24]:
# Limpieza de datos
# Estandarizar nombres de departamentos, municipios, valores categoricos de Sexo y valores de la columna Estado. 
clean_data.loc[:,"Nombre departamento"] = clean_data["Nombre departamento"].str.upper()
clean_data.loc[:,"Nombre municipio"] = clean_data["Nombre municipio"].str.upper()
clean_data.loc[:,"Sexo"] = clean_data["Sexo"].str.upper()
clean_data.loc[(clean_data["Estado"] == "leve") | (clean_data["Estado"] == "LEVE"),"Estado"] = "Leve"

# Remover valores N/A de la columna estado
clean_data = clean_data[~clean_data["Estado"].isna()]
clean_data.shape

# Remover valores invalidos de Unidad de medida de edad
clean_data = clean_data[clean_data["Unidad de medida de edad"]<=3]
clean_data.shape

(6349701, 9)

In [25]:
# Construcción de nuevos datos
# Categorización asintomaticos
clean_data.loc[~clean_data["Fecha de inicio de síntomas"].isna(),"Fecha de inicio de síntomas"] = 'No'
clean_data.loc[clean_data["Fecha de inicio de síntomas"].isna(),"Fecha de inicio de síntomas"] = 'Si'
#clean_data["Fecha de inicio de síntomas"].value_counts()

# Modificación de edad basado en la unidad de medida de edad
clean_data.loc[clean_data["Unidad de medida de edad"] == 2,"Edad"] = clean_data["Edad"]/12.0
clean_data.loc[clean_data["Unidad de medida de edad"] == 3,"Edad"] = clean_data["Edad"]/365.0

# Creación de rango de edad
clean_data.loc[clean_data["Edad"] < 5,"rango_edad"] = "0 a 4 años"
clean_data.loc[(clean_data["Edad"] >= 5) & (clean_data["Edad"] < 12),"rango_edad"] = "5 a 11 años"
clean_data.loc[(clean_data["Edad"] >= 12) & (clean_data["Edad"] < 26),"rango_edad"] = "12 a 25 años"
clean_data.loc[(clean_data["Edad"] >= 26) & (clean_data["Edad"] < 41),"rango_edad"] = "26 a 40 años"
clean_data.loc[(clean_data["Edad"] >= 41) & (clean_data["Edad"] < 64),"rango_edad"] = "41 a 64 años"
clean_data.loc[clean_data["Edad"] >= 64,"rango_edad"] = "65 años y más"

C:\Users\crist\AppData\Local\Temp\ipykernel_3848\1790464415.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.5        0.25       0.5        ... 0.91666667 0.5        0.16666667]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  clean_data.loc[clean_data["Unidad de medida de edad"] == 2,"Edad"] = clean_data["Edad"]/12.0
C:\Users\crist\AppData\Local\Temp\ipykernel_3848\1790464415.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0 a 4 años' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  clean_data.loc[clean_data["Edad"] < 5,"rango_edad"] = "0 a 4 años"


In [26]:
# Integracion de los datos
departamentos = pd.read_csv('departamentos.csv')
municipios = pd.read_csv('municipios.csv')

#clean_data.merge(municipios, left_on='Nombre municipio', right_on='Homologo')
clean_data = clean_data.merge(departamentos[['Nombre','Población DANE','Categoría','Homologo']], left_on='Nombre departamento', right_on='Homologo')

In [27]:
# Formato de datos 
# Remover columnas innecesarias
clean_data = clean_data.drop(columns=["Unidad de medida de edad","Nombre departamento","Homologo"])
# Renombrar columnas a formato camel_case
clean_data = clean_data.rename(columns={
    "ID de caso": "id_caso",
    "Nombre": "departamento",
    "Nombre municipio": "municipio",
    "Edad": "edad",
    "Sexo": "sexo",
    "Tipo de contagio": "tipo_contagio",
    "Estado": "estado",
    "Fecha de inicio de síntomas":"asintomatico",
    "Población DANE":"poblacion_departamento",
    "Categoría":"categoria_departamento"
    })

# Formatear categoria de departamento como String
clean_data['categoria_departamento'] = clean_data['categoria_departamento'].astype(str)


In [31]:
clean_data = clean_data[['id_caso','departamento','municipio','poblacion_departamento','categoria_departamento','edad','sexo','tipo_contagio','asintomatico','rango_edad','estado']]

In [33]:
# Exportar datos
clean_data.to_csv("covid19_clean_data.csv")